# 모듈 

In [1]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import datetime
import time as t
import pandas as pd
from urllib import request as req
import math
import sys
import fileIO as io

In [2]:
def addZero(isLowerTen):
    for index in range(len(isLowerTen)):
        if isLowerTen[index] < 10:
            isLowerTen[index] = '0' + str(isLowerTen[index])
        else:
            isLowerTen[index] = str(isLowerTen[index])
    return isLowerTen

# case1) 현재 실시간 검색어 크롤링

In [ ]:
nowDateTime = datetime.datetime.now()
year = nowDateTime.year
month = nowDateTime.month
day = nowDateTime.day
hour = nowDateTime.hour
minute = nowDateTime.minute
isLowerTen = [month, day, hour, minute]
month, day, hour, minute = addZero(isLowerTen)

chrome_driver = 'D:/bigdata/Python_workspace/jupyter/chromedriver.exe'
driver = webdriver.Chrome(chrome_driver)
url = 'https://datalab.naver.com/keyword/realtimeList.naver'
url += f'?datetime={year}-{month}-{day}T{hour}%3A{minute}%3A00&where=main'
driver.get(url)
soup = bs(driver.page_source,'html.parser')
driver.close()
# soup

# case2) 하루 실시간 검색어 크롤링

In [4]:
soup_list = []
chrome_driver = 'D:/bigdata/Python_workspace/jupyter/chromedriver.exe'
# open window
driver = webdriver.Chrome(chrome_driver)
nowDateTime = datetime.datetime.now()
year = nowDateTime.year
month = nowDateTime.month
day = nowDateTime.day
isLowerTen = [month, day]
month, day = addZero(isLowerTen)

times = pd.date_range('05:00:00',f'17:00:00',freq='1min')
for time in times:
    hour = time.hour
    minute = time.minute
    hour, minute = addZero([hour,minute])
    url = 'https://datalab.naver.com/keyword/realtimeList.naver'
    url += f'?datetime={year}-{month}-{day}T{hour}%3A{minute}%3A00'
    driver.get(url)
    soup_list.append(bs(driver.page_source,'html.parser'))
    t.sleep(0.1)
driver.close()
io.list_to_csv('naverRtop20_'+f'{year}-{month}-{day}', soup_list)
len(soup_list)

# case3) 2020년 실시간 검색어 크롤링
## 트래픽 제한으로 인하여 실행이 제한됨

In [7]:
# chrome_driver = 'D:/bigdata/Python_workspace/jupyter/chromedriver.exe'
# # open window
# driver = webdriver.Chrome(chrome_driver)
# nowDateTime = datetime.datetime.now()
# year = nowDateTime.year
# month = nowDateTime.month
# day = nowDateTime.day
# hour = nowDateTime.hour
# minute = nowDateTime.minute
# isLowerTen = [month, day, hour, minute]
# month, day, hour, minute = addZero(isLowerTen)
# dates = pd.date_range('2020-01-01', '2020-06-26', freq="D")
# for date in dates[::-1]:
#     soup_list = []
#     year = date.year
#     month = date.month
#     day = date.day
#     if f'{month}{day}' == f'{nowDateTime.month}{nowDateTime.day}':
#         times = pd.date_range('00:00:00',f'{hour}:{minute}:00',freq='1min')
#     else:
#         times = pd.date_range('00:00:00','23:59:00',freq='1min')
#     year,month,day = addZero([year, month, day])
#     for time in times:
#         hour = time.hour
#         minute = time.minute
#         hour, minute = addZero([hour,minute])
#         url = 'https://datalab.naver.com/keyword/realtimeList.naver'
#         url += f'?datetime={year}-{month}-{day}T{hour}%3A{minute}%3A00'
#         print(url)
#         driver.get(url)
#         soup_list.append(bs(driver.page_source,'html.parser'))
#         t.sleep(2)
#     io.list_to_csv('naverRtop20_'+f'{year}-{month}-{day}', soup_list)
# driver.close()

# 주가 관련 검색어 필터링

In [26]:
soup_list = io.csv_to_list('naverRtop20_2020-07-06')
soup_list = list(map(lambda i:bs(i,'html.parser'), soup_list))

In [27]:
times = pd.date_range('00:00:00',f'{18}:00:00',freq='1min')
ranking_list = list(map(lambda i:i.select_one('.ranking_box'), soup_list))
col = [f'rank{i}' for i in range(1,21)]
df_ranking = pd.DataFrame(index=times, columns=col)
df_rel_rank = pd.DataFrame(index=times, columns=col)
for ranking_box, date in zip(ranking_list, times):
    item_title = ranking_box.select('.item_title')
    item_title_list = list(map(lambda i:i.text, item_title))
    df_ranking.loc[date] = item_title_list
    item_sub_list_raw = list(map(lambda i:i.find_next_sibling(), item_title))
    item_sub_list = []
    for item_sub in item_sub_list_raw:
        if item_sub:
            item_sub_list.append(list(map(lambda i:i.replace('\n','').strip(), item_sub.text.split(','))))
        else:
            item_sub_list.append('')
    df_rel_rank.loc[date] = item_sub_list

# 상장법인목록 가져오기

In [28]:
url = 'http://kind.krx.co.kr/corpgeneral/corpList.do'
url += '?currentPageSize=5000'
url += '&fiscalYearEnd=all'
url += '&location=all'
url += '&method=download'
url += '&orderMode=3'
url += '&orderStat=D'
url += '&pageIndex=1'
url += '&searchType=13'
df = pd.read_html(url, header=0)
jongmok_list = list(df[0]['회사명'])

In [29]:
df_ranking = df_ranking[df_ranking.isin(jongmok_list)]
set_ranking = set()
for time in times:
    for ranking in df_ranking:
        data = df_ranking[ranking][time]
#         print(ranking, date)
        if type(data) is str:
            if data not in map(lambda i:i[0], set_ranking):
                set_ranking.add((data,str(time)))
#                 print(data)
#                 print(data, time)

In [30]:
io.list_to_csv("rtop20_stocks.csv",list(set_ranking))

In [14]:
df_sample = df[0].loc[:, ['회사명','종목코드','업종']]

In [21]:
df[0].loc[df[0]['회사명']=='마인', :]

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역


In [18]:
df_real = df[0]

In [19]:
df_real.columns

Index(['회사명', '종목코드', '업종', '주요제품', '상장일', '결산월', '대표자명', '홈페이지', '지역'], dtype='object')